## Part-of-Speech Tagging using Hidden Markov Model (HMM)

In [ ]:
import nltk
import numpy as np
from collections import defaultdict
from nltk.corpus import treebank

In [ ]:
# Download treebank dataset
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
def train_hmm(corpus):
    # Initialize dictionaries for counts
    tag_counts = defaultdict(int)
    transition_counts = defaultdict(lambda: defaultdict(int))
    emission_counts = defaultdict(lambda: defaultdict(int))
    vocab = set()

    # Process corpus with universal tagset
    for sentence in corpus:
        prev_tag = '<START>'
        tag_counts['<START>'] += 1
        for word, tag in sentence:
            word = word.lower()
            tag_counts[tag] += 1
            transition_counts[prev_tag][tag] += 1
            emission_counts[tag][word] += 1
            vocab.add(word)
            prev_tag = tag
        transition_counts[prev_tag]['<END>'] += 1
        tag_counts['<END>'] += 1

    # Compute probabilities with add-one smoothing
    tags = list(tag_counts.keys())
    vocab_size = len(vocab)
    transition_probs = defaultdict(lambda: defaultdict(float))
    emission_probs = defaultdict(lambda: defaultdict(float))

    for tag in tags:
        # Transition probabilities
        total_trans = sum(transition_counts[tag].values())
        for next_tag in tags:
            transition_probs[tag][next_tag] = (transition_counts[tag][next_tag] + 1) / (total_trans + len(tags))

        # Emission probabilities with <UNK> handling
        total_emissions = sum(emission_counts[tag].values())
        for word in vocab:
            emission_probs[tag][word] = (emission_counts[tag][word] + 1) / (total_emissions + vocab_size + 1)
        emission_probs[tag]["<UNK>"] = 1 / (total_emissions + vocab_size + 1)

    return tags, vocab, transition_probs, emission_probs


In [ ]:
def viterbi(sentence, tags, vocab, transition_probs, emission_probs):
    V = [{}]
    path = {}

    # Initialize with start state
    for tag in tags:
        if tag not in {'<START>', '<END>'}:
            word = sentence[0].lower()
            emission = emission_probs[tag].get(word, emission_probs[tag]["<UNK>"])
            V[0][tag] = np.log(transition_probs['<START>'][tag]) + np.log(emission)
            path[tag] = [tag]

    # Run Viterbi
    for t in range(1, len(sentence)):
        V.append({})
        new_path = {}
        word = sentence[t].lower()
        for curr_tag in tags:
            if curr_tag not in {'<START>', '<END>'}:
                emission = emission_probs[curr_tag].get(word, emission_probs[curr_tag]["<UNK>"])
                max_prob, best_prev_tag = max(
                    (V[t-1][prev_tag] + np.log(transition_probs[prev_tag][curr_tag]) + np.log(emission), prev_tag)
                    for prev_tag in tags if prev_tag not in {'<START>', '<END>'}
                )
                V[t][curr_tag] = max_prob
                new_path[curr_tag] = path[best_prev_tag] + [curr_tag]
        path = new_path

    # Termination
    max_prob, best_tag = max(
        (V[-1][tag] + np.log(transition_probs[tag]['<END>']), tag)
        for tag in tags if tag not in {'<START>', '<END>'}
    )
    return path[best_tag], max_prob


In [ ]:
def evaluate_hmm(test_corpus, tags, vocab, transition_probs, emission_probs):
    correct = 0
    total = 0
    for sentence in test_corpus:
        words = [word for word, _ in sentence]
        true_tags = [tag for _, tag in sentence]
        predicted_tags, _ = viterbi(words, tags, vocab, transition_probs, emission_probs)
        correct += sum(1 for true, pred in zip(true_tags, predicted_tags) if true == pred)
        total += len(true_tags)
    return correct / total

In [ ]:
# Load data with universal tagset
corpus = treebank.tagged_sents(tagset='universal')[:3000]  # Training
test_corpus = treebank.tagged_sents(tagset='universal')[3000:3750]  # Testing

# Train HMM
tags, vocab, transition_probs, emission_probs = train_hmm(corpus)

In [ ]:
# Sample input/output
sample_sentence = ["The", "quick", "brown", "fox", "jumps"]
predicted_tags, prob = viterbi(sample_sentence, tags, vocab, transition_probs, emission_probs)
print("Sample Input:", sample_sentence)
print("Predicted Tags:", predicted_tags)

Sample Input: ['The', 'quick', 'brown', 'fox', 'jumps']
Predicted Tags: ['DET', 'ADJ', 'NOUN', 'NOUN', '.']


In [ ]:
# Evaluate
accuracy = evaluate_hmm(test_corpus, tags, vocab, transition_probs, emission_probs)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.8832
